# import requests
import pandas as pd
import numpy as np
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from pandas.io.json import json_normalize
import json
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import time

In [2]:
URL = 'https://dolphin.jump-technology.com:8443/api/v1/' 
AUTH = ('ESPRIT_GROUPE3', 'KquAg2TDrpUv57Hr') 
def columns_to_str(columns): 
    if isinstance(columns, list):
        k="?columns="
        for a in columns:
            k=k+a+"&columns="
        k = k[:-9]
        return k
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    payload = {'date': date }
    res = requests.get(URL + endpointApi + columns_to_str(columns),
    params=payload,
    auth=AUTH,
    verify=False) 
    return res.content.decode('utf-8')

In [3]:
def getAssetRatios( ratio=[],asset=[],star_date='2013-06-13',end_date='2019-04-18', full_response=False, columns=list()):
    payload = {'ratio': ratio,'asset':asset,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
     headers={'Content-Type': 'application/x-www-form-urlencoded'},
    auth=AUTH,
    verify=False) 
    k = res.content.decode('utf-8')
    k= json.loads(k)
    kk = pd.DataFrame(columns=['ASSET_DATABASE_ID','ratio_id','ratioValue','ratioType'])
    for a in asset:
        for b in ratio:
            kk = kk.append(
                pd.DataFrame([ (a,b, str(k[str(a)][str(b)]["value"]),str(k[str(a)][str(b)]["type"]) )],columns=['ASSET_DATABASE_ID','ratio_id','ratioValue','ratioType']),
                ignore_index=True)
    return kk

In [ ]:
def getDisponibleRatios():
    res = requests.get(URL + "ratio" ,
    auth=AUTH,
    verify=False)
    return json_normalize(json.loads(res.content.decode('utf-8')))

In [ ]:
def getAllAssets():
    col = list(["ASSET_DATABASE_ID","LABEL","TYPE","LAST_CLOSE_VALUE","CURRENCY"])
    assets = get_data('asset',columns=col)
    jdata = json.loads(assets)
    assets = pd.DataFrame.from_dict(json_normalize(jdata), orient='columns')
    assets = assets[['ASSET_DATABASE_ID.value','LABEL.value','LAST_CLOSE_VALUE.value','TYPE.value',"CURRENCY.value"]]
    assets.columns =["ASSET_DATABASE_ID","LABEL","LAST_CLOSE_VALUE",'TYPE',"CURRENCY"]
    return assets

In [4]:
def getAssetCotation(asset_id):
    col = list(["date","asset","nav","gross","pl","close","return"])
    assetC = get_data('asset/'+str(asset_id)+'/quote?start_date=2013-06-14&end_date=2019-04-18')
    jdata = json.loads(assetC)
    assetC = pd.DataFrame.from_dict(json_normalize(jdata), orient='columns')
    assetC = assetC[["asset.value","date.value","nav.value","gross.value","pl.value","close.value","return.value"]]
    assetC.columns =["asset","date","nav","gross","pl","close","return"]
    return assetC

In [5]:
def getAllRatiosForAsset(asset): 
    ratios = getAssetRatios(ratio=[13,9,12,14,10,8],asset=[asset])["ratioValue"].tolist()
    return pd.Series(ratios[0],ratios[1],ratios[2],ratios[3],ratios[4],ratios[5]).astype(np.float)

In [6]:
def getAssetWithRAtios(assets):
    assetswithRAtio = assets.apply(lambda x : (pd.Series(getAssetRatios(ratio=[13,9,12,14,10,8],asset=[str(x["ASSET_DATABASE_ID"])])["ratioValue"].tolist())),axis =1)
    assetswithRAtio.columns = ['Rendement', 'RendementAnnualise', 'Sharpe','VaR_Historique','Volatilite','RDT']
    assetswithRAtio = assets.join(assetswithRAtio)
    return assetswithRAtio

In [7]:
def getAssetRendementAnnualisee( asset=[],ratio=[13],star_date='2013-06-14',end_date='2019-04-18', full_response=False, columns=list()):
    kk=pd.DataFrame()
    l = list()
    payload = {'ratio': ratio,'asset':asset,'benchmark':1,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
    headers={'Content-Type': 'application/x-www-form-urlencoded'},auth=AUTH,verify=False) 
    k = res.content.decode('utf-8')
    k=json.loads(k)
    for a in asset:
        l.append(str(k[str(a)]["13"]["value"]))
    kk=pd.DataFrame(l,index=asset)
    kk[kk.columns] = kk[kk.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return kk

In [8]:
def getAssetCorrelation( asset=[],ratio=[11],star_date='2013-06-14',end_date='2019-04-18', full_response=False, columns=list()):
    kk = pd.DataFrame()
    l = list()
    for i in asset:
        payload = {'ratio': ratio,'asset':asset,'benchmark':i,'start_date': star_date,'end_date':end_date }
        res = requests.post(URL + "ratio/invoke" ,
        data=str(payload),
         headers={'Content-Type': 'application/x-www-form-urlencoded'},
        auth=AUTH,
        verify=False) 
        k = res.content.decode('utf-8')
        k=json.loads(k)
        ls=list()
        for a in asset:
            ls.append(str(k[str(a)][str(11)]["value"]))
        l.append(ls)
    kk=pd.DataFrame(l,index=asset,columns=asset)
    kk[kk.columns] = kk[kk.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return kk

In [9]:
def getAssetVolatilite( asset=[],ratio=[10],star_date='2013-06-14',end_date='2019-04-18', full_response=False, columns=list()):
    kk=pd.DataFrame()
    l = list()
    payload = {'ratio': ratio,'asset':asset,'benchmark':1,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
    headers={'Content-Type': 'application/x-www-form-urlencoded'},auth=AUTH,verify=False) 
    k = res.content.decode('utf-8')
    k=json.loads(k)
    for a in asset:
        r=1
        l.append(str(k[str(a)]["10"]["value"]))
    kk=pd.DataFrame(l,index=asset)
    kk[kk.columns] = kk[kk.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return kk

In [10]:
def is_nan(x):
    return (x is np.nan or x != x)

In [11]:
def getAssetCovariance(asset=[]):
    cor=getAssetCorrelation(asset)
    vol=getAssetVolatilite(asset)
    for i in cor.columns:
        for j in cor.columns:
            cor[i][j]=cor[i][j]*np.sqrt(vol[0][i]*vol[0][j])
          
    return cor

In [12]:
def getAssetChangeCur(assets):
    l = list()
    for i in assets.CURRENCY.to_list():
        result = get_data('currency/rate/'+str(i)+'/to/EUR?date=2013-06-14')
        value = json.loads(result)['rate']['value'].replace(',','.')
        value = float(value)
        l.append(value)
    return l

In [13]:
def elimineNanFromCov(S):
    for a in S.index.to_list():
        for b in S.columns.to_list():
            if(is_nan(S[str(b)][str(a)])):
                S[str(b)][str(a)]=0
    return S

In [14]:
def getAssetCotation(asset_id):
    col = list(["date","asset","nav","gross","pl","close","return"])
    assetC = get_data('asset/'+str(asset_id)+'/quote?start_date=2013-06-14&end_date=2019-04-18')
    jdata = json.loads(assetC)
    assetC = pd.DataFrame.from_dict(json_normalize(jdata), orient='columns')
    assetC = assetC[["asset.value","date.value","nav.value","gross.value","pl.value","close.value","return.value"]]
    assetC.columns =["asset","date","nav","gross","pl","close","return"]
    return assetC

In [15]:
def coursPeriodique(Fasset):
    assetC=None
    for a in Fasset.index.tolist():
        assetC2 = getAssetCotation(a)
        assetC2 = assetC2[["date","close"]]
        assetC2 = assetC2.set_index('date')
        assetC2.columns = [str(a)]
        assetC = pd.concat([assetC,assetC2], axis=1,sort=False)
    assetC[assetC.columns] = assetC[assetC.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return assetC

In [16]:
def coursAction(asset):
    l=list()
    for a in asset.columns.to_list():
        t=None
        df = asset[a].to_list()
        for i in df:
            if (is_nan(i)==False):
                l.append(i)
                break
    return l

In [17]:
def cleanDataToPush(Fasset):
    df=pd.DataFrame()
    df["asset"]=Fasset.ASSET_DATABASE_ID
    df["quantity"]=Fasset.quantity
    return df

In [18]:
def jsonpush(df):
    b=pd.DataFrame()
    b=pd.concat([b,df.asset],axis=1,sort=False)
    b=pd.concat([b,df.quantity],axis=1,sort=False)
    b=b.T
    a='{"label":"ESPRIT_PTF_3","currency":{"code": "EUR"},"type": "front","values": {"2013-06-14": ['
    for i in b.columns:
        a=a+'{"asset" :'+b[i].to_json()+'},'
    a=a[0:-1]
    a=a+"]}}"
    return a

In [19]:
def push(df):
    payload = jsonpush(df)
    res = requests.put(URL + "portfolio/1840/dyn_amount_compo" ,
    data=payload,
     headers={'Content-Type': 'application/x-www-form-urlencoded'},
    auth=AUTH,
    verify=False) 
    return 

In [20]:
def getPortfolioSharpe( asset=[1840],ratio=[12],star_date='2013-06-14',end_date='2019-04-18', full_response=False):
    payload = {'ratio': ratio,'asset':asset,'benchmark':0,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
    headers={'Content-Type': 'application/x-www-form-urlencoded'},auth=AUTH,verify=False) 
    k = res.content.decode('utf-8')
    k=json.loads(k)
    k=float(k[str(1840)]["12"]["value"].replace(',','.'))
    return k

In [21]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : 'Total'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        'Total', ascending=False).round(1)
        return mis_val_table_ren_columns

In [22]:
def portfolioWeights(df):    
    start_time = time.time()
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    # Optimise for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.095),gamma=1)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    return cleaned_weights

In [23]:
def getcleanhistoric():    
    start_time = time.time()
    assets = getAllAssets().dropna(0)
    actifs = assets[assets.TYPE != "INDEX"]
    actif=actifs.ASSET_DATABASE_ID.to_list()
    assetC=None
    for a in actifs.ASSET_DATABASE_ID.tolist():
        assetC2 = getAssetCotation(a)
        assetC2 = assetC2[["date","close"]]
        assetC2 = assetC2.set_index('date')
        assetC2.columns = [str(a)]
        assetC = pd.concat([assetC,assetC2], axis=1,sort=False)
    df=assetC
    df[df.columns] = df[df.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    df=df.iloc[0:-247].sort_index(axis=0)
    N=missing_values_table(df);
    GAct=(N[N.Total<45]).index
    df=df[GAct]
    return df

In [44]:
def test(df):
    a= portfolioWeights(df)
    a=pd.DataFrame.from_dict(a,orient='index')
    a.columns=["Weights"]
    b=a[a.Weights>0.0095]
    b.Weights=b.Weights/b.Weights.sum();
    b=b.sort_values(by='Weights',ascending=False);
    assets = getAllAssets().dropna(0)
    assets.index=assets.ASSET_DATABASE_ID
    Fasset=assets.T[b.index.to_list()].T
    Fasset["CHANGE"]=getAssetChangeCur(Fasset)
    Fasset["Weights"]=b.Weights
    Fasset["CoursOC"]=coursAction(coursPeriodique(Fasset))
    Fasset["quantity"]=round(Fasset["Weights"]*100000000000/(Fasset["CHANGE"]*Fasset["CoursOC"]));
    if(Fasset.Weights[Fasset.TYPE=="STOCK"].sum()>0.5):    
        push(cleanDataToPush(Fasset))
        push(cleanDataToPush(Fasset))
        return Fasset

In [ ]:
df=getcleanhistoric()
cc=test(df)
for i in range(2):
    nass=cc.ASSET_DATABASE_ID.count()
    waug=0.004
    sharp=getPortfolioSharpe()
    for a in range(nass):
        i=nass-1-a
        b=False
        cc1=cc
        
        push(cleanDataToPush(cc))
        push(cleanDataToPush(cc))

        while(cc1.Weights[i]+waug/(cc1.Weights.sum()+waug)<0.1):

            cc1.Weights[i]=cc1.Weights[i]+waug
            cc1["quantity"]=round(cc1["Weights"]*10000000/(cc1["CHANGE"]*cc1["CoursOC"]));
            push(cleanDataToPush(cc1))
            push(cleanDataToPush(cc1))
        
            if(sharp-getPortfolioSharpe()>0):
                cc1.Weights[i]=cc1.Weights[i]-waug
                if(b==False):
                
                    while(cc1.Weights[i]-waug>0):
                        cc1.Weights[i]=cc1.Weights[i]-waug
                        cc1["quantity"]=round(cc1["Weights"]*10000000/(cc1["CHANGE"]*cc1["CoursOC"]));
                        push(cleanDataToPush(cc1))
                        push(cleanDataToPush(cc1))
                        
                        if(sharp-getPortfolioSharpe()>0):
                            push(cleanDataToPush(cc))
                            push(cleanDataToPush(cc))   
                            break
                        
                        cc=cc1
                        sharp=getPortfolioSharpe()
              
                push(cleanDataToPush(cc))
                push(cleanDataToPush(cc))
                break
        
            cc=cc1
            b=True
            sharp=getPortfolioSharpe()
        
        push(cleanDataToPush(cc))
        push(cleanDataToPush(cc))

In [ ]:
push(cleanDataToPush(cc1[cc1.Weights>0.01]))